# *Fine-tuning* de *Large Language Models* (LLMs)

<p align="center">
  <a href="https://colab.research.google.com/github/auduvignac/llm-finetuning/blob/main/notebooks/project/finetuning-projet.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Ouvrir dans Google Colab"/>
  </a>
</p>

Le but de ce projet est de réaliser le *fine-tuning* d'un LLM.


## Installation des bibliothèques/libraires requises

In [ ]:
!wget -q https://raw.githubusercontent.com/auduvignac/llm-finetuning/refs/heads/main/setup_env.py -O setup_env.py
%run setup_env.py

In [ ]:
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

import contextlib
import math

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from datasets import (
    load_dataset,
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from tabulate import (
    tabulate,
)
from torch.utils.data import (
    DataLoader,
)
from tqdm import (
    tqdm,
)

# Si le notebook est exécuté dans un environnement jupyter, la librairie
# ci-dessus peut être utilisée
from tqdm.notebook import (
    tqdm,
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling,
    DistilBertConfig,
    DistilBertModel,
    DistilBertTokenizer,
    Trainer,
    TrainingArguments,
)

# Utilisation d’un GPU avec CUDA lorsque disponible sur la machine d’exécution.
# L’utilisation d’un GPU pour l’apprentissage entraîne souvent d’énormes
# accélérations lors de l’entraînement.
# Voir https://developer.nvidia.com/cuda-downloads pour installer CUDA
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

In [ ]:
class LlamaFineTuner:
    def __init__(
        self, model_id="TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
    ):
        self.model_id = model_id

        # Config quantization 4-bit
        self.bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype="float16",
            bnb_4bit_use_double_quant=False,
        )

        # Chargement du tokenizer et modèle
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_id)
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_id,
            quantization_config=self.bnb_config,
            device_map="auto",
        )
        self.dataset = None
        self.trainer = None

    def __str__(self):
        return str(self.model)

    def generate(
        self,
        prompt: str,
        max_new_tokens: int = 200,
        temperature: float = 1.0,
        instruction_mode: bool = False,
    ):
        """
        Génèreation de texte avec le modèle.

        Si instruction_mode=True, formate le prompt en mode
        Instruction/Response.
        """

        DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

        # Formatage du prompt
        if instruction_mode:
            text = f"### Instruction:\n{prompt}\n\n### Response:\n"
        else:
            text = prompt

        inputs = self.tokenizer(text, return_tensors="pt").to(DEVICE)

        # Correction du warning: désactivation du checkpointing et
        # réactivation du cache
        with contextlib.suppress(Exception):
            self.model.gradient_checkpointing_disable()
            self.model.config.use_cache = True

        outputs = self.model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
        )
        return self.tokenizer.decode(outputs[0])

    def prepare_dataset(self, dataset_name="tatsu-lab/alpaca"):
        def process_data(sample):
            return self.tokenizer(sample["text"])

        data = load_dataset(dataset_name)
        self.dataset = data.map(process_data, batched=True)
        return self.dataset

    def prepare_for_kbit_training(self):
        self.model = prepare_model_for_kbit_training(self.model)

    def print_trainable_parameters(self):
        trainable_params = 0
        all_param = 0
        for _, param in self.model.named_parameters():
            all_param += param.numel()
            if param.requires_grad:
                trainable_params += param.numel()
        print(
            f"trainable params: {trainable_params} || all params: {all_param} "
            f"|| trainable%: {100 * trainable_params / all_param:.2f}"
        )

    def apply_lora(self):
        config = LoraConfig(
            r=8,
            lora_alpha=32,
            target_modules=["q_proj", "k_proj", "v_proj"],
            lora_dropout=0.05,
            bias="none",
            task_type="CAUSAL_LM",
        )
        self.model = get_peft_model(self.model, config)
        self.print_trainable_parameters()

    def train(self, output_dir="outputs", max_steps=100):
        self.tokenizer.pad_token = self.tokenizer.eos_token

        training_args = TrainingArguments(
            per_device_train_batch_size=16,
            gradient_accumulation_steps=1,
            max_steps=max_steps,
            learning_rate=2e-4,
            fp16=True,
            logging_steps=1,
            output_dir=output_dir,
            optim="paged_adamw_8bit",
            report_to="none",
        )

        self.trainer = Trainer(
            model=self.model,
            train_dataset=self.dataset["train"],
            args=training_args,
            data_collator=DataCollatorForLanguageModeling(
                self.tokenizer, mlm=False
            ),
        )

        self.model.config.use_cache = False
        self.trainer.train()

    def workflow(
        self,
        quick_test_prompt: str = "Paris is the capital of",
        instruction_prompt: str = "Propose an outdoor activity.",
        max_steps: int = 10,
    ):
        """
        Exécution d'un workflow complet :
          1. Génération initiale (avant entraînement)
          2. Préparation du dataset
          3. Préparation du modèle pour k-bit training
          4. Application de LoRA
          5. Entraînement (max_steps configurables)
          6. Génération finale en mode instruction

        Args:
            quick_test_prompt: Texte de génération avant entraînement
            instruction_prompt: Instruction pour la génération finale
            max_steps: Nombre d'étapes d'entraînement
        """
        print(" Génération initiale (avant fine-tuning)…")
        print(self.generate(quick_test_prompt, max_new_tokens=50))

        print("\n Préparation du dataset…")
        self.prepare_dataset()

        print("\n Préparation k-bit training…")
        self.prepare_for_kbit_training()

        print("\n Application de LoRA…")
        self.apply_lora()

        print("\n Entraînement…")
        self.train(max_steps=max_steps)

        print("\nGénération finale (après fine-tuning)…")
        print(
            self.generate(
                instruction_prompt, instruction_mode=True, max_new_tokens=100
            )
        )

In [ ]:
llm = LlamaFineTuner()
llm.workflow()